In [1]:
import os
from ensure import ensure_annotations

In [2]:
os.chdir("../")

In [3]:
!pwd

/Users/ngkuissi/Dev/Image_Search_Engine


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    command: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from imageSearchEngine.constants import *
from imageSearchEngine.utils.file_helpers import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    @ensure_annotations
    def __init__(
        self,
        config_filepath:Path = CONFIG_FILE_PATH,
        params_filepath:Path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        #self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    @ensure_annotations
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            command=config.command,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )
        return data_ingestion_config

In [7]:
import os
import zipfile
from imageSearchEngine.logging.logger import log
from imageSearchEngine.exception import CustomException

In [8]:
class DataIngestion:
    '''
    class that would for data ingestion of the required data to the system
    '''
    @ensure_annotations
    def __init__(self, config: DataIngestionConfig):
        '''
        requires: config  and creates the object that would get the data
        '''
        self.config = config
    
    def download_data(self):
        '''
        using the information provided from the config file the method downloads the data to the specificed zip
        returns none
        '''
        path = self.config.local_data_file
        dir = self.config.root_dir
        if not os.path.exists(path):
            os.system(self.config.command + f' -p {dir}')
            log.info(f'successfully downloaded the data from kaggle at {path}')
        else:
            log.info('file already existed download not needed')
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        log.info('done unziping the data')

    

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise CustomException(e)

[2023-09-10 17:17:10,083: INFO: file_helpers: yaml file: config/config.yaml loaded successfully]
[2023-09-10 17:17:10,084: INFO: file_helpers: created directory at: artifacts]
[2023-09-10 17:17:10,085: INFO: file_helpers: created directory at: artifacts/data_ingestion]


 92%|█████████▏| 216M/235M [00:03<00:00, 65.9MB/s] 


[2023-09-10 17:17:14,568: INFO: 2722938969: successfully downloaded the data from kaggle at artifacts/data_ingestion/tinyimagenettorch.zip]


100%|██████████| 235M/235M [00:03<00:00, 66.1MB/s]


[2023-09-10 17:17:27,537: INFO: 2722938969: done unziping the data]


In [16]:
!pwd

/Users/ngkuissi/Dev/Image_Search_Engine/research


In [20]:
!pwd

/Users/ngkuissi/Dev/Image_Search_Engine
